In [1]:
%%cython
# Declare rank and genus
r = 5; r_ = 2; g = 2
dim = (r**2-1) * (g-1) + 1   # dimension of M x C
dim_ = r_**2 * (g-1) + 2    # dimension of N x C

In [13]:
from sage.all import *

class Curve:
    """ 
        EXAMPLES::

        We write down the basis of the cohomology of C

            sage: g = 2
            sage: C = Curve(g)
            sage: R = PolynomialRing(QQ, 'h').fraction_field()
            sage: R.inject_variables()
            Defining h
            sage: H = C.cohomology_ring(R)
            sage: H.inject_variables()
            Defining sigma_1, tau_1, sigma_2, tau_2, pt
            sage: expr = h * sigma_1 * tau_1 - h * pt
            sage: expr
            h*sigma_1*tau_1 - h*pt
            sage: def basis_up_to(A, n):
            sage:    return [A.basis(k) for k in range(n)]
            sage: basis_up_to(H,3)
            [[1], [sigma_1, sigma_2, tau_1, tau_2], [pt]]

        And a the generators for its rank 3 universal tautological ring 

            sage: r = 3
            sage: L = C.universal_tautological_ring(R, r)
            sage: L.inject_variables()
            sage: c_2τ1*c_2σ2
            -c_2σ2*c_2τ1
    """
    def __init__(self, g):
        self.g = g
    
    def cohomology_ring(self, R):
        g = self.g
        generators =  [f'sigma_{i}' for i in range(1, g+1)] + [f'tau_{i}' for i in range(1, g+1)] + ['pt']
        degrees = [1] * (2 * g) + [2]
        A = GradedCommutativeAlgebra(R, generators, degrees=degrees)
        A.inject_variables()
        I_1 = set(eval(f'sigma_{i}*sigma_{j}') for i in range(1, g+1) for j in range(1, g+1))
        I_2 = set(eval(f'tau_{i}*tau_{j}') for i in range(1, g+1) for j in range(1, g+1))
        I_3 = set(eval(f'sigma_{i}*tau_{j} - pt') for i in range(1, g+1) for j in range(1, g+1))

        H = A.quotient(A.ideal(list(I_1 | I_2 | I_3)))
        return H
    
    def __repr__(self):
        return f"Curve of genus {self.g}."

    def universal_tautological_ring(self,R, r): 
        """
        Returns the universal tautological ring of rank r objects with base ring R
        """
        generators = [f'c_{k}one' for k in range(1, r+1)] + [f'c_{k}σ{i}' for k in range(1, r+1) for i in range(1, self.g+1)]+ [f'c_{k}τ{i}' for k in range(1, r+1) for i in range(1, self.g+1)] + [f'c_{k}pt' for k in range(2, r+1)]
        degrees = [2*k for k in range(1, r+1)] + [2*k-1 for k in range(1, r+1) for i in range(1, self.g+1)] + [2*k-1 for k in range(1, r+1) for i in range(1, self.g+1)] + [2*k -2 for k in range(2, r+1)]
        return GradedCommutativeAlgebra(R, generators, degrees=degrees)
        
    
class Chains:
    def __init__(self, g, a):
        self.g = g
        self.a = a
        self.length = len(a)

    def total_rank(self):
        return sum(alpha[0] for alpha in self.a)
    
    def universal_tautological_ring(self,R):
        return Curve(self.g).universal_tautological_ring(R, self.total_rank())
    
    def factor_ring_gens_dict(self,R,h):
        """ 
        Returns a dictionary d for the generators of the factor ring. We want 
        d[(i, γ, j, k)] to be the generator c_iγjF_k of the factor ring. 

        EXAMPLES:: 

            sage: g = 2
            sage: a = [(2, 1), (1, 1)]
            sage: Ch = Chains(g, a)
            sage: R = PolynomialRing(QQ, 'h').fraction_field()
            sage: R.inject_variables()
            sage: A = Ch.tautological_factor_ring(R, h)
            sage: d = Ch.factor_ring_gens_dict(R, h)
            sage: d[(2, 'σ', 2, 1)]
            c_2σ2F_1
        """
        gensdict = self.tautological_factor_ring(R, h).gens_dict()
        one_dict = { (k, 'one', j) : gensdict[f'c_{k}oneF_{j}'] for j in range(1, self.length + 1) for k in range(1, self.a[j-1][0] + 1) }
        σ_dict = { (k, 'σ', i, j) : gensdict[f'c_{k}σ{i}F_{j}'] for j in range(1, self.length + 1) for k in range(1, self.a[j-1][0] + 1) for i in range(1, self.g + 1) }
        τ_dict = { (k, 'τ', i, j) : gensdict[f'c_{k}τ{i}F_{j}'] for j in range(1, self.length + 1) for k in range(1, self.a[j-1][0] + 1) for i in range(1, self.g + 1) }
        pt_dict = { (k, 'pt', j) : gensdict[f'c_{k}ptF_{j}'] for j in range(1, self.length + 1) for k in range(2, self.a[j-1][0] + 1) }
        return {**one_dict, **σ_dict, **τ_dict, **pt_dict}

    def tautological_factor_ring(self,R, h):
        """
        Return the universal tautological ring of the product of moduli of bundles of the terms in the chain. The index label c_kγF_j represents 
        1(x)...(x)c_k(γ)(x)...(x) 1 (x) 1
        with the non-identity class in the jth position. 

        INPUT:

        - ``R`` -- a base ring
        - ``h`` -- an element of ``R`` (the equivariant parameter)

        EXAMPLES::
            sage: g = 2
            sage: a = [(2, 1), (1, 1)]
            sage: Ch = Chains(g, a)
            sage: R = PolynomialRing(QQ, 'h').fraction_field()
            sage: R.inject_variables()
            Defining h
            sage: A = Ch.tautological_factor_ring(R, h)
            Defining ...
            sage: A.inject_variables()
            sage: h*c_1σ1F_2*c_2oneF_1
            h*c_2oneF_1*c_1σ1F_2
        """
        l = self.length
        rk = lambda j : self.a[j][0]
        gone = [f'c_{k}oneF_{j}' for j in range(1,l + 1) for k in range(1, rk(j-1)+1) ]
        gσ = [f'c_{k}σ{i}F_{j}' for j in range(1,l + 1) for k in range(1, rk(j-1)+1) for i in range(1, self.g+1)]
        gτ = [f'c_{k}τ{i}F_{j}' for j in range(1,l + 1) for k in range(1, rk(j-1)+1) for i in range(1, self.g+1)]
        gpt = [f'c_{k}ptF_{j}' for j in range(1,l + 1) for k in range(2, rk(j-1)+1)]
        generators =  gone + gσ + gτ + gpt
        done = [2*k for j in range(1,l + 1) for k in range(1, rk(j-1)+1)]
        dσ = [2*k-1 for j in range(1,l + 1) for k in range(1, rk(j-1)+1) for i in range(1, self.g+1)]
        dτ = [2*k-1 for j in range(1,l + 1) for k in range(1, rk(j-1)+1) for i in range(1, self.g+1)]
        dpt = [2*k -2 for j in range(1,l + 1) for k in range(2, rk(j-1)+1)]
        degrees = done + dσ + dτ + dpt
        return GradedCommutativeAlgebra(R, generators, degrees=degrees)

    @staticmethod 
    def c_E_tensorh(cE, h):
        """ 
        Expresses the list of chern classes of E(x)L in terms of h (=c_1(L)) and cE = [c_0(E), c_1(E),..., c_r(E)]. Formula is 
        c_i(E(x)L)= sum_{a=0}^i binomial(r-i + a, a) c_{i-a}(E)h^a

        EXAMPLES :: 

            sage: g = 2
            sage: a = [(2, 1), (1, 1)]
            sage: Ch = Chains(g, a)
            sage: C = Curve(g)
            sage: R = PolynomialRing(QQ, 'h').fraction_field()
            sage: R.inject_variables()
            Defining h
            sage: A = C.universal_tautological_ring(R,4)
            sage: cE = (1,) + A.gens()[0:4]
            sage: Ch.c_E_tensorh(cE, 2*h)
            [1,
            c_1one + (8*h),
            c_2one + (6*h)*c_1one + (24*h^2),
            c_3one + (4*h)*c_2one + (12*h^2)*c_1one + (32*h^3),
            c_4one + (2*h)*c_3one + (4*h^2)*c_2one + (8*h^3)*c_1one + (16*h^4)]
         """
        r = len(cE)-1
        return [sum(binomial(r-i + a, a) * cE[i-a] * h^a for a in range(i+1)) for i in range(r+1)]

    def to_factor(self, x):
        """ 
        Takes a tautological ring expression x and writes it in terms of the factor tautological ring generators. 
        Uses that the universal bundle is E = E_0(+)hE_1(+)h^2E_2(+)...(+)h^nE_n
        and so c(E) = c(E_1)c(hE_2)c(h^3E_3)...c(h^nE_n)
        """    
        raise NotImplementedError
        pass

    def __repr__(self):
        return f"Moduli stack of chains of class {self.a} on a curve of genus {self.g}."
    
    @staticmethod
    def TensorProductRings(As):
        g = lambda i,j: As[i].gen(j)
        indices = cartesian_product([range(len(As[i].gens())) for i in range(len(As))])
        label = lambda idx : 'x'.join([str(g(i,idx[i])) for i in range(len(As))])
        gens = [label(idx) for idx in indices]
        deg_idx = lambda idx : sum(g(i,idx[i]).degree() for i in range(len(As))) 
        degrees = [deg_idx(idx) for idx in indices]
        R = GradedCommutativeAlgebra(QQ, gens, degrees=degrees)
        return R 

g = 2
a = [(2, 1), (1, 1)]
Ch = Chains(g, a)
C = Curve(g)
R = PolynomialRing(QQ, 'h').fraction_field()
R.inject_variables()
A = C.universal_tautological_ring(R,4)
cE = (1,) + A.gens()[0:4]
Ch.c_E_tensorh(cE,2*h)

Ch.tautological_factor_ring(R,h)

Defining h


Graded Commutative Algebra with generators ('c_1oneF_1', 'c_2oneF_1', 'c_1oneF_2', 'c_1σ1F_1', 'c_1σ2F_1', 'c_2σ1F_1', 'c_2σ2F_1', 'c_1σ1F_2', 'c_1σ2F_2', 'c_1τ1F_1', 'c_1τ2F_1', 'c_2τ1F_1', 'c_2τ2F_1', 'c_1τ1F_2', 'c_1τ2F_2', 'c_2ptF_1') in degrees (2, 4, 2, 1, 1, 3, 3, 1, 1, 1, 1, 3, 3, 1, 1, 2) over Fraction Field of Univariate Polynomial Ring in h over Rational Field

In [3]:
binomial(5,2)
factorial(5)/(factorial(2)*factorial(3))

10

In [5]:
%display plain
r = 3
C = Curve(g) 
R = PolynomialRing(QQ, 'h').fraction_field()
L = C.universal_tautological_ring(R, r)
L.inject_variables()
c_2τ1*c_2σ2

Defining c_1one, c_2one, c_3one, c_1σ1, c_1σ2, c_2σ1, c_2σ2, c_3σ1, c_3σ2, c_1τ1, c_1τ2, c_2τ1, c_2τ2, c_3τ1, c_3τ2, c_2pt, c_3pt


-c_2σ2*c_2τ1

In [6]:
g = 2
C = Curve(g)
R = PolynomialRing(QQ, 'h').fraction_field()
R.inject_variables()
H = C.cohomology_ring(R)
H.inject_variables()
h*sigma_1*tau_1 - h*pt
def basis_up_to(A, n):
    return [A.basis(k) for k in range(n)]
basis_up_to(H,3)

Defining h
Defining sigma_1, sigma_2, tau_1, tau_2, pt
Defining sigma_1, sigma_2, tau_1, tau_2, pt


[[1], [sigma_1, sigma_2, tau_1, tau_2], [pt]]

In [ ]:


g = 2
C = Curve(g)
R = PolynomialRing(QQ, 'h').fraction_field()
R.inject_variables()
H = C.cohomology_ring(R)
A = C.universal_tautological_ring(R,4)
TensorProductRings([A,H])

Defining h
Defining sigma_1, sigma_2, tau_1, tau_2, pt


Graded Commutative Algebra with generators ('c_1onexsigma_1', 'c_1onexsigma_2', 'c_1onextau_1', 'c_1onextau_2', 'c_1onexpt', 'c_2onexsigma_1', 'c_2onexsigma_2', 'c_2onextau_1', 'c_2onextau_2', 'c_2onexpt', 'c_3onexsigma_1', 'c_3onexsigma_2', 'c_3onextau_1', 'c_3onextau_2', 'c_3onexpt', 'c_4onexsigma_1', 'c_4onexsigma_2', 'c_4onextau_1', 'c_4onextau_2', 'c_4onexpt', 'c_1σ1xsigma_1', 'c_1σ1xsigma_2', 'c_1σ1xtau_1', 'c_1σ1xtau_2', 'c_1σ1xpt', 'c_1σ2xsigma_1', 'c_1σ2xsigma_2', 'c_1σ2xtau_1', 'c_1σ2xtau_2', 'c_1σ2xpt', 'c_2σ1xsigma_1', 'c_2σ1xsigma_2', 'c_2σ1xtau_1', 'c_2σ1xtau_2', 'c_2σ1xpt', 'c_2σ2xsigma_1', 'c_2σ2xsigma_2', 'c_2σ2xtau_1', 'c_2σ2xtau_2', 'c_2σ2xpt', 'c_3σ1xsigma_1', 'c_3σ1xsigma_2', 'c_3σ1xtau_1', 'c_3σ1xtau_2', 'c_3σ1xpt', 'c_3σ2xsigma_1', 'c_3σ2xsigma_2', 'c_3σ2xtau_1', 'c_3σ2xtau_2', 'c_3σ2xpt', 'c_4σ1xsigma_1', 'c_4σ1xsigma_2', 'c_4σ1xtau_1', 'c_4σ1xtau_2', 'c_4σ1xpt', 'c_4σ2xsigma_1', 'c_4σ2xsigma_2', 'c_4σ2xtau_1', 'c_4σ2xtau_2', 'c_4σ2xpt', 'c_1τ1xsigma_1', 'c_1τ1x

In [8]:
generators = ['sigma_1tau_2', 'sigma_1tau_1']
degrees = [1, 1]
A = GradedCommutativeAlgebra(QQ, generators, degrees=degrees)
A.inject_variables()
A.gen(0)

Defining sigma_1tau_2, sigma_1tau_1


sigma_1tau_2

In [182]:
%display latex
lnames = ['c_kτ_1F1', 'c_kτ_1F2']
R = GradedCommutativeAlgebra(QQ,lnames, degrees = [2,2]) 
R.inject_variables()
c_kτ_1F1*c_kτ_1F2

Defining c_kτ_1F1, c_kτ_1F2


c_kτ_1F1*c_kτ_1F2

In [ ]:
from sage.all import *


Moduli stack of chains of class [(1, 2), (2, 3), (0, 1)] on a curve of genus 2.


In [ ]:
# %%cython
from sage.all import *
# Define the ring generators a, b, f, x, y, z

def generators(r,g):
    names_a = ["a"+str(i) for i in range(2,r+1)]
    names_b = ["b"+str(i)+str(j) for i in range(2,r+1) for j in range(1,2*g+1)]
    names_f = ["f"+str(i) for i in range(2,r+1)]
    return names_a + names_b + names_f

def generators_(r_,g):
    names_x = ["x"+str(i) for i in range(2,r_+1)]
    names_y = ["y"+str(i)+str(j) for i in range(1,r_+1) for j in range(1,2*g+1)]
    names_z = ["z"+str(i) for i in range(2,r_+1)]
    return names_x + names_y + names_z
    
# Declare the cohomological degrees of a, b, f, x, y, z

def deg(u):
    if u[0] == 'a' or u[0] == 'z':
        degree = 2*int(u[1])
    elif u[0] == 'b' or u[0] == 'y':
        degree = 2*int(u[1])-1
    elif u[0] == 'f' or u[0] == 'x':
        degree = 2*int(u[1])-2
    return degree
    
# List of degrees of a, b, f and x, y, z as above

def deg_M(r,g): return list(deg(u) for u in generators(r,g))

def deg_N(r_,g): return list(deg(u) for u in generators_(r_,g))

# Define classes e in H^1 of the curve

# Convention follows BLM: e1 and e2 are (0,1)-classes, e3 and e4 are (1,0)-classes

def loops(g):
    names_e = list(set("e"+str(i) for i in range(1,2*g+1)))
    return names_e

# List of degrees of e classes

def deg_C(g): return list(1 for e in loops(g))
# Define a ring that contains all a, b, f, e classes; think of this as M tensor C

# w is the point class of C

R = GradedCommutativeAlgebra(QQ, list(generators(r,g)) + generators_(r_,g) + list(loops(g)) + list('w'), degrees = deg_M(r,g) + deg_N(r_,g) + deg_C(g) + [2])
# Some notations

name_list = [str(j) for j in R.gens()]
index = {i: u for i, u in zip(name_list, R.gens())}
# Give explicit names to ring generators

locals().update(index)
# List of all classes

gens_list = list(index.values())
degree_list = [gens.degree() for gens in gens_list]
# Input by hand the ideal for the curve

I_1 = set(eval('e'+str(i)) * eval('e'+str(j)) for i in range(1, 2*g+1) for j in range(i+1, 2*g+1))
I_2 = set(eval('e'+str(i)) * eval('e'+str(i+g)) for i in range(1, g+1))
I_3 = set(eval('e'+str(i)) * eval('e'+str(i+g)) - w for i in range(1, g+1))

# !!! this needs to be slightly modified for r=2 and g=5: change e5*e10 - w to e10*e5-w

I = R.ideal(list(I_1 - I_2) + list(I_3))

In [70]:
R.<h> = LaurentPolynomialRing(QQ)
h^2

h^2

In [74]:
gens = ['σ', 'τ']
A = GradedCommutativeAlgebra(R, gens, degrees=[1,1])

TypeError: cannot call Singular function 'nc_algebra' with ring parameter of type '<class 'sage.rings.polynomial.multi_polynomial_ring.MPolynomialRing_polydict_domain_with_category'>'

In [99]:
R

Univariate Polynomial Ring in h over Rational Field

In [109]:
R = singular.ring('(0,h)', '(x)', 'dp').__dir__()
# A = GradedCommutativeAlgebra(R, ['σ'], degrees=[1])
# R.set_ring()
R

['__add__',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getmetaclass__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__pari__',
 '__pos__',
 '__pow__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_act_on_',
 '_acted_upon_',
 '_add_',
 '_ascii_art_',
 '_axiom_',
 '_axiom_init_',
 '_

In [111]:
R = singular.ring(0, '(h)', 'lp')
R.set_ring()
R

polynomial ring, over a field, global ordering
// coefficients: QQ
// number of vars : 1
//        block   1 : ordering lp
//                  : names    h
//        block   2 : ordering C

In [133]:
R = PolynomialRing(QQ,'h').fraction_field()
R.inject_variables()
A = GradedCommutativeAlgebra(R, ['σ', 'τ', 'pt'], degrees=[1,1,2])
I = A.ideal([A.gens()[0]*A.gens()[1] - A.gens()[2]])
H = A.quotient(I)
H.inject_variables()
h*σ*τ

Defining h
Defining σ, τ, pt


h*pt